In [1]:
import pandas as pd 
import numpy as np
from scipy.sparse import load_npz,save_npz,lil_matrix,csr_matrix
from tqdm import tqdm 
import os

In [2]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [3]:
y.shape

(422185, 2963)

In [4]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_weights.csv'
weights = pd.read_csv(path)

In [5]:
weights

,task_id,task_type,training_weight,aggregation_weight
0,0,OTHER,1.000000,0.0
1,1,OTHER,0.500000,0.0
2,2,OTHER,0.500000,0.0
3,3,OTHER,0.333333,0.0
4,4,OTHER,0.333333,1.0
...,...,...,...,...
2958,2958,AUX_HTS,0.100000,0.0
2959,2959,AUX_HTS,0.100000,0.0
2960,2960,AUX_HTS,0.100000,0.0
2961,2961,AUX_HTS,0.100000,0.0


In [6]:
# link this to an id in the summary eps 0.05 file

In [7]:
path = '../../data/Mellody_tuner/baseline_plus_aux_data/baseline_plus_aux_data/results_tmp/classification/T8c.csv'
t8c = pd.read_csv(path)

In [8]:
t8c.columns

Index(['cont_classification_task_id', 'classification_task_id',
       'input_assay_id', 'assay_type', 'variance_quorum_OK', 'is_auxiliary',
       'use_in_regression', 'threshold', 'threshold_method', 'direction',
       'training_quorum_OK', 'evaluation_quorum_OK', 'aggregation_weight',
       'weight', 'num_total_actives', 'num_fold_min_actives',
       'num_total_inactives', 'num_fold_min_inactives', 'n_tasks'],
      dtype='object')

In [10]:
path = './files/baseline_plus_aux_data/input/T0_image_pseudolabels_plus_baseline.csv'
t0 = pd.read_csv(path)

In [11]:
df_mgd = pd.merge(
            weights
            ,
            pd.merge(
                t0
                ,t8c
                ,how='inner'
                ,on='input_assay_id'
            )
            ,how='inner'
            ,left_on='task_id'
            ,right_on='cont_classification_task_id'
        )

In [12]:
path = './cp/summary_eps_0.05.csv'
df_cp = pd.read_csv(path)

In [13]:
df_mgd_cp = pd.merge(
    df_cp
    ,df_mgd
    ,how='inner'
    ,left_on='index'
    ,right_on='melloddy_col_idx'
)

In [14]:
path = './cp/summary_eps_0.05_mgd_cp.csv'
df_mgd_cp.to_csv(path, index=False)

In [16]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [17]:
y_lil = lil_matrix(y)
all_cctis = set(df_mgd_cp['cont_classification_task_id'])
base_path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/confidence_selection/'
for i,l1 in tqdm(enumerate(lst)):
    for j,l2 in enumerate(lst): 
        file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
        y_out = y_lil.copy()
        cctis_to_keep = set(df_mgd_cp.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'])
        cctis_to_drop = all_cctis.difference(cctis_to_keep)
        for ccti in cctis_to_drop: 
            y_out[:,ccti] = 0
        save_npz(os.path.join(base_path,'cls_T10_y_'+file_core+'.npz')
                 ,csr_matrix(y_out))

10it [05:36, 33.69s/it]
